In [1]:
# for validation
import numpy as np
import pandas as pd
import datetime
from netCDF4 import Dataset,num2date,date2num
import gpflow as gf
from eofs.standard import Eof
import time
from datetime import datetime, timedelta
import numpy.ma as ma
import os
from typing import Sequence
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# Load historical simulation data
historical_simulation_data = Dataset('D:\CWatM-new\Brahmaputra\output230\discharge_daily_mri.nc','r')

# Load historical observation data
historical_observation_data = Dataset('E:\data\era5_runoff\disdata\era5_data1979-2014_5min.nc','r')

mri_hist_train_origin = historical_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

obsdata_origin = historical_observation_data.variables['dis24'][:]
historical_observation_data.close()
historical_simulation_data.close()

mask_t = mri_hist_train_origin == mri_hist_train_origin[1][0][0]
mri_hist_train = ma.array(mri_hist_train_origin,mask = mask_t)
obsdata = ma.array(obsdata_origin,mask = mask_t)
#数据切片调试-可以不做
#mri_hist_train_old = ma.array(mri_hist_train_origin,mask = mask_t)
#obsdata_old = ma.array(obsdata_origin,mask = mask_t)
#mri_hist_train = mri_hist_train_old[13000:][:][:]
#obsdata = obsdata_old[13000:][:][:]

In [3]:
#提取两个数据的主成分，减少分析两个数据差异的复杂度
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
#coslat = np.cos(np.deg2rad(lats)).clip(0., 1.)
#wgts = np.sqrt(coslat)[..., np.newaxis]


# Perform EOF analysis on data1 and data2
#obsdata_solver = Eof(obsdata, weights=None, center=False)
obsdata_solver = Eof(obsdata, weights=None,center=False)

#explain for how much variance
obsdata_mode = obsdata_solver.varianceFraction(neigs=4)
print(obsdata_mode)

# Retrieve the leading EOFs for both datasets
obsdata_eofs = obsdata_solver.eofs(neofs=4)
print(obsdata_eofs.shape)

#variance correlation
obsdata_eofc = obsdata_solver.eofsAsCorrelation(neofs=4)


#create pseudo-ECs for GPR

#源代码中obsdata 不存在 obsdata -> obsdata_old
mri_hist_train_ECs = obsdata_solver.projectField(mri_hist_train,neofs = 4)
obsdata_ECs = obsdata_solver.projectField(obsdata,neofs = 4)
print(mri_hist_train_ECs.shape)

[0.9868846  0.0065986  0.00354106 0.00170928]
(4, 89, 189)
(13149, 4)


In [26]:
#将两列ECs各自做为训练集
mri_hist_train_ECs_l1 = mri_hist_train_ECs[:,0].reshape(13149,1)
mri_hist_train_ECs_x1 = tf.convert_to_tensor(mri_hist_train_ECs_l1)
mri_hist_train_ECs_x1=tf.cast(mri_hist_train_ECs_x1,tf.float64)
mri_hist_train_ECs_l2 = mri_hist_train_ECs[:,1].reshape(13149,1)
mri_hist_train_ECs_x2 = tf.convert_to_tensor(mri_hist_train_ECs_l2)
mri_hist_train_ECs_x2=tf.cast(mri_hist_train_ECs_x2,tf.float64)
mri_hist_train_ECs_l3 = mri_hist_train_ECs[:,2].reshape(13149,1)
mri_hist_train_ECs_x3 = tf.convert_to_tensor(mri_hist_train_ECs_l3)
mri_hist_train_ECs_x3=tf.cast(mri_hist_train_ECs_x3,tf.float64)
mri_hist_train_ECs_l4 = mri_hist_train_ECs[:,3].reshape(13149,1)
mri_hist_train_ECs_x4 = tf.convert_to_tensor(mri_hist_train_ECs_l4)
mri_hist_train_ECs_x4=tf.cast(mri_hist_train_ECs_x4,tf.float64)
obsdata_ECs_l1 = obsdata_ECs[:,0].reshape(13149,1)
obsdata_ECs_y1 = tf.convert_to_tensor(obsdata_ECs_l1)
obsdata_ECs_y1=tf.cast(obsdata_ECs_y1,tf.float64)
obsdata_ECs_l2 = obsdata_ECs[:,1].reshape(13149,1)
obsdata_ECs_y2 = tf.convert_to_tensor(obsdata_ECs_l2)
obsdata_ECs_y2=tf.cast(obsdata_ECs_y2,tf.float64)
obsdata_ECs_l3 = obsdata_ECs[:,2].reshape(13149,1)
obsdata_ECs_y3 = tf.convert_to_tensor(obsdata_ECs_l3)
obsdata_ECs_y3=tf.cast(obsdata_ECs_y3,tf.float64)
obsdata_ECs_l4 = obsdata_ECs[:,3].reshape(13149,1)
obsdata_ECs_y4 = tf.convert_to_tensor(obsdata_ECs_l4)
obsdata_ECs_y4=tf.cast(obsdata_ECs_y4,tf.float64)

In [27]:
#EOF1
#standardize x,y
n_input_ECs = 4
scaler_x = StandardScaler()
scaler_y = StandardScaler()
mri_train = np.c_[mri_hist_train_ECs_x1,mri_hist_train_ECs_x2,mri_hist_train_ECs_x3,mri_hist_train_ECs_x4]
mri_train_sc = scaler_x.fit_transform(mri_train)
#inducing（修改）
n_inducing_points=700
model_mri_eof = list()
obs_train_all = np.c_[obsdata_ECs_y1,obsdata_ECs_y2,obsdata_ECs_y3,obsdata_ECs_y4]
for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mri_train_sc[:,0].min(), mri_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(mri_train_sc[:, j].min(), mri_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mri_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mri_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mri_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_mri_eof[i].training_loss, model_mri_eof[i].trainable_variables)
    gf.utilities.print_summary(model_mri_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_mri_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_mri_eof1_timeperiod:',opt_mri_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_mri_eof[i].predict_y(mri_train_sc, full_cov=False)
    mri_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mri_pred_mean) / np.mean(obs_train)
    y = (np.std(mri_pred_mean) / np.mean(mri_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.79656
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.61402
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.21163
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.55033, -4.27024, -6.50628..."


The  1  ECs training for obsdata
opt_mri_eof1_timeperiod: 59703.28099999999
0.9981978317708251 0.8795451304717558


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.12862
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.56679
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.40451
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.7309, -4.24245, -6.53226..."


The  2  ECs training for obsdata
opt_mri_eof1_timeperiod: 59731.562999999995
1.0459129312265811 0.7140966061096639


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.63883
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.45662
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.70744
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.49815, -4.21643, -6.73137..."


The  3  ECs training for obsdata
opt_mri_eof1_timeperiod: 49474.28100000002
0.9749035718051298 0.4890299564879788


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.18546
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.56817
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.61724
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.70852, -4.07724, -6.59503..."


The  4  ECs training for obsdata
opt_mri_eof1_timeperiod: 36566.516
1.0099785788519886 0.5744476871567876


In [28]:
mri_pred_ECs = np.zeros((13149,n_input_ECs))
mri_pred_ECs_lb = np.zeros((13149,n_input_ECs))
mri_pred_ECs_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

for i in range(n_input_ECs):
    y_pred_mean, y_pred_var= model_mri_eof[i].predict_y(mri_train_sc, full_cov=False)

    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    print(y_pred_mean,obs_train_sc)
    mri_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    print(mri_pred_mean,obs_train)
    B = np.mean(mri_pred_mean) / np.mean(obs_train)
    y = (np.std(mri_pred_mean) / np.mean(mri_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)
    mri_pred_ECs[:,i] =np.squeeze(mri_pred_mean)
    print(mri_pred_ECs[:,i])
    y__pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    mri_pred_ECs_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    mri_pred_ECs_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
mri_hist_train_ECs_SGPRcor=tf.concat([mri_pred_ECs[:,0].reshape(13149, 1),mri_pred_ECs[:,1].reshape(13149, 1),mri_pred_ECs[:,2].reshape(13149, 1),mri_pred_ECs[:,3].reshape(13149, 1)],axis=1)
mri_hist_train_ECs_lb_SGPRcor=tf.concat([mri_pred_ECs_lb[:,0].reshape(13149, 1),mri_pred_ECs_lb[:,1].reshape(13149, 1),mri_pred_ECs_lb[:,2].reshape(13149, 1),mri_pred_ECs_lb[:,3].reshape(13149, 1)],axis=1)
mri_hist_train_ECs_ub_SGPRcor=tf.concat([mri_pred_ECs_ub[:,0].reshape(13149, 1),mri_pred_ECs_ub[:,1].reshape(13149, 1),mri_pred_ECs_ub[:,2].reshape(13149, 1),mri_pred_ECs_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(mri_hist_train_ECs_SGPRcor,columns = ['mri_EC1','mri_EC2','mri_EC3','mri_EC4'])
df_lb_merge = pd.DataFrame(mri_hist_train_ECs_lb_SGPRcor,columns = ['mri_EC1_lb','mri_EC2_lb','mri_EC3_lb','mri_EC4_lb'])
df_ub_merge = pd.DataFrame(mri_hist_train_ECs_ub_SGPRcor,columns = ['mri_EC1_ub','mri_EC2_ub','mri_EC3_ub','mri_EC4_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

tf.Tensor(
[[0.85867372]
 [0.86613392]
 [0.87447384]
 ...
 [0.85993235]
 [0.86115105]
 [0.86293259]], shape=(13149, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[[-37327.08482382]
 [-36288.61380745]
 [-35127.68317489]
 ...
 [-37151.88211239]
 [-36982.23710934]
 [-36734.2443761 ]] [[-26780.41015625]
 [-26305.3046875 ]
 [-25864.76757812]
 ...
 [-31726.390625  ]
 [-31462.87109375]
 [-31136.0703125 ]]
0.9981978317708251 0.8795451304717558
[-37327.08482382 -36288.61380745 -35127.68317489 ... -37151.88211239
 -36982.23710934 -36734.2443761 ]
tf.Tensor(
[[-0.03502159]
 [-0.03655524]
 [-0.04018728]
 ...
 [ 0.04852154]
 [ 0.04716731]
 [ 0.0460797 ]], shape=(13149, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[[ -973.73818114]
 [-1000.03148175]
 [-1062.30055096]
 ...
 [  458.55599361]
 [  435.33849905]
 [  416.69208537]] [[-2039.22070312]
 [-1982.69091797]
 [-1931

In [34]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
mri_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
mri_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
mri_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_mri_eof_2GP = list()
mri_pred_ECs_range = np.zeros((13149,n_input_ECs))
for i in range(n_input_ECs):
    #变成误差值
    mri_pred_ECs_range[:,i] = np.subtract(mri_pred_ECs_ub[:,i],mri_pred_ECs_lb[:,i])
    mri_train_2GP = np.c_[mri_pred_ECs[:,i],mri_pred_ECs_lb[:,i],mri_pred_ECs_ub[:,i],mri_pred_ECs_range[:,i] ]
    mri_train_2GP_sc = scaler_z.fit_transform(mri_train_2GP)
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mri_train_2GP_sc[:,0].min(), mri_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(mri_train_2GP_sc[:, j].min(), mri_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mri_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mri_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mri_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_mri_eof_2GP[i].training_loss, model_mri_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_mri_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_mri_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_mri_eof1_2GP_timeperiod:',opt_mri_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_mri_eof_2GP[i].predict_y(mri_train_2GP_sc, full_cov=False)
    mri_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mri_pred_mean) / np.mean(obs_train)
    y = (np.std(mri_pred_mean) / np.mean(mri_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    mri_pred_ECs_2GP[:,i] =np.squeeze(mri_pred_mean)
    print(mri_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    mri_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    mri_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,115.30188
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,30.22162
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.19236
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.57052, -3.21439, -3.417..."


The  1  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 36566.516
[-38322.00702416 -37399.84857451 -36368.99600434 ... -38182.98488061
 -38032.40916061 -37812.04734038]
0.9999981343963706 0.8986403737895664


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,329.94482
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,75.25286
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.36423
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.78863, -5.34912, -2.61164..."


The  2  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 36566.516
[-1134.77965954 -1163.11737825 -1229.87893424 ...   397.21517042
   372.3079353    352.30197814]
1.0000423893384622 0.7972333314149451


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,7.36388
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,7.0459
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.61903
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.67712, -5.7941, -6.17529..."


The  3  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 36566.516
[3214.08154667 3195.83011645 3168.16578578 ... 2867.57849061 2800.99326904
 2747.48303171]
1.000116736610136 0.6167424894196456


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,518.97673
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,113.62564
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.56485
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-0.96417, -6.50656, -10.16497..."


The  4  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 36566.516
[495.77112854 341.1497573  164.23254572 ... 499.0855855  444.59534248
 396.04278019]
1.000005342291969 0.6595103324697047


In [35]:
#mri_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
#mri_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

#for i in range(n_input_ECs):
#    y_pred_mean, y_pred_var= model_mri_eof_2GP[i].predict_y(mri_train_2GP_sc, full_cov=False)
#    obs_train = obs_train_all[:,i].reshape(13149,1)
#    obs_train_sc = scaler_y.fit_transform(obs_train)
#    print(y_pred_mean,obs_train_sc)
#    mri_pred_mean_2GP = scaler_y.inverse_transform(y_pred_mean)
#    print(mri_pred_mean_2GP,obs_train)
#    B = np.mean(mri_pred_mean_2GP) / np.mean(obs_train)
#    y = (np.std(mri_pred_mean_2GP) / np.mean(mri_pred_mean_2GP)) / (np.std(obs_train) / np.mean(obs_train))
#    print(B,y)
#    mri_pred_ECs_2GP[:,i] =np.squeeze(mri_pred_mean_2GP)
#    print(mri_pred_ECs_2GP[:,i])
#    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
#    y_pred_lb = y_pred_mean - y__pred_conf
#    y_pred_ub = y_pred_mean + y__pred_conf
#    mri_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
#    mri_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
mri_hist_train_ECs_2GP_SGPRcor=tf.concat([mri_pred_ECs_2GP[:,0].reshape(13149, 1),mri_pred_ECs_2GP[:,1].reshape(13149, 1),mri_pred_ECs_2GP[:,2].reshape(13149, 1),mri_pred_ECs_2GP[:,3].reshape(13149, 1)],axis=1)
mri_hist_train_ECs_2GP_lb_SGPRcor=tf.concat([mri_pred_ECs_2GP_lb[:,0].reshape(13149, 1),mri_pred_ECs_2GP_lb[:,1].reshape(13149, 1),mri_pred_ECs_2GP_lb[:,2].reshape(13149, 1),mri_pred_ECs_2GP_lb[:,3].reshape(13149, 1)],axis=1)
mri_hist_train_ECs_2GP_ub_SGPRcor=tf.concat([mri_pred_ECs_2GP_ub[:,0].reshape(13149, 1),mri_pred_ECs_2GP_ub[:,1].reshape(13149, 1),mri_pred_ECs_2GP_ub[:,2].reshape(13149, 1),mri_pred_ECs_2GP_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(mri_hist_train_ECs_2GP_SGPRcor,columns = ['mri_EC1_2GP','mri_EC2_2GP','mri_EC3_2GP','mri_EC4_2GP'])
df_lb_merge = pd.DataFrame(mri_hist_train_ECs_2GP_lb_SGPRcor,columns = ['mri_EC1_2GP_lb','mri_EC2_2GP_lb','mri_EC3_2GP_lb','mri_EC4_2GP_lb'])
df_ub_merge = pd.DataFrame(mri_hist_train_ECs_2GP_ub_SGPRcor,columns = ['mri_EC1_2GP_ub','mri_EC2_2GP_ub','mri_EC3_2GP_ub','mri_EC4_2GP_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean_2GP.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb_2GP.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub_2GP.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

In [36]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((13149,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [37]:
recondata_SGPR_mri_mix4_2GP = ReconstructDataFromEOFandECs(obsdata_eofs,mri_hist_train_ECs_2GP_SGPRcor)

In [38]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(1979, 1, 1)
    end_date = datetime(2014, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 1979-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [39]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc = out_dir+'recondata_SGPR_mri_mix4_2GP_weighted_nomean.nc' 
writeNetCDF(recondata_SGPR_mri_mix4_2GP,out_file_nc) 

# 做未来预测

In [43]:
#读入数据
ssp126_mri_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_mri_ssp126.nc','r')
ssp370_mri_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_mri_ssp370.nc','r')
ssp585_mri_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_mri_ssp585.nc','r')

mri_ssp126_train_origin = ssp126_mri_simulation_data.variables['discharge'][:]
mri_ssp370_train_origin = ssp370_mri_simulation_data.variables['discharge'][:]
mri_ssp585_train_origin = ssp585_mri_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

mask_t = mri_ssp126_train_origin == mri_ssp126_train_origin[1][0][0]

mri_ssp126_train = ma.array(mri_ssp126_train_origin,mask = mask_t)
mri_ssp370_train = ma.array(mri_ssp370_train_origin,mask = mask_t)
mri_ssp585_train = ma.array(mri_ssp585_train_origin,mask = mask_t)

mri_ssp126_train_ECs = obsdata_solver.projectField(mri_ssp126_train,neofs = 4)
mri_ssp370_train_ECs = obsdata_solver.projectField(mri_ssp370_train,neofs = 4)
mri_ssp585_train_ECs = obsdata_solver.projectField(mri_ssp585_train,neofs = 4)

ssp126_mri_simulation_data.close()
ssp370_mri_simulation_data.close()
ssp585_mri_simulation_data.close()

In [44]:
#将两列ECs各自做为训练集
mri_ssp126_train_ECs_l1 = mri_ssp126_train_ECs[:,0].reshape(28854,1)
mri_ssp126_train_ECs_x1 = tf.convert_to_tensor(mri_ssp126_train_ECs_l1)
mri_ssp126_train_ECs_x1=tf.cast(mri_ssp126_train_ECs_x1,tf.float64)
mri_ssp126_train_ECs_l2 = mri_ssp126_train_ECs[:,1].reshape(28854,1)
mri_ssp126_train_ECs_x2 = tf.convert_to_tensor(mri_ssp126_train_ECs_l2)
mri_ssp126_train_ECs_x2=tf.cast(mri_ssp126_train_ECs_x2,tf.float64)
mri_ssp126_train_ECs_l3 = mri_ssp126_train_ECs[:,2].reshape(28854,1)
mri_ssp126_train_ECs_x3 = tf.convert_to_tensor(mri_ssp126_train_ECs_l3)
mri_ssp126_train_ECs_x3=tf.cast(mri_ssp126_train_ECs_x3,tf.float64)
mri_ssp126_train_ECs_l4 = mri_ssp126_train_ECs[:,3].reshape(28854,1)
mri_ssp126_train_ECs_x4 = tf.convert_to_tensor(mri_ssp126_train_ECs_l4)
mri_ssp126_train_ECs_x4=tf.cast(mri_ssp126_train_ECs_x4,tf.float64)
mri_ssp370_train_ECs_l1 = mri_ssp370_train_ECs[:,0].reshape(28854,1)
mri_ssp370_train_ECs_x1 = tf.convert_to_tensor(mri_ssp370_train_ECs_l1)
mri_ssp370_train_ECs_x1=tf.cast(mri_ssp370_train_ECs_x1,tf.float64)
mri_ssp370_train_ECs_l2 = mri_ssp370_train_ECs[:,1].reshape(28854,1)
mri_ssp370_train_ECs_x2 = tf.convert_to_tensor(mri_ssp370_train_ECs_l2)
mri_ssp370_train_ECs_x2=tf.cast(mri_ssp370_train_ECs_x2,tf.float64)
mri_ssp370_train_ECs_l3 = mri_ssp370_train_ECs[:,2].reshape(28854,1)
mri_ssp370_train_ECs_x3 = tf.convert_to_tensor(mri_ssp370_train_ECs_l3)
mri_ssp370_train_ECs_x3=tf.cast(mri_ssp370_train_ECs_x3,tf.float64)
mri_ssp370_train_ECs_l4 = mri_ssp370_train_ECs[:,3].reshape(28854,1)
mri_ssp370_train_ECs_x4 = tf.convert_to_tensor(mri_ssp370_train_ECs_l4)
mri_ssp370_train_ECs_x4=tf.cast(mri_ssp370_train_ECs_x4,tf.float64)
mri_ssp585_train_ECs_l1 = mri_ssp585_train_ECs[:,0].reshape(28854,1)
mri_ssp585_train_ECs_x1 = tf.convert_to_tensor(mri_ssp585_train_ECs_l1)
mri_ssp585_train_ECs_x1=tf.cast(mri_ssp585_train_ECs_x1,tf.float64)
mri_ssp585_train_ECs_l2 = mri_ssp585_train_ECs[:,1].reshape(28854,1)
mri_ssp585_train_ECs_x2 = tf.convert_to_tensor(mri_ssp585_train_ECs_l2)
mri_ssp585_train_ECs_x2=tf.cast(mri_ssp585_train_ECs_x2,tf.float64)
mri_ssp585_train_ECs_l3 = mri_ssp585_train_ECs[:,2].reshape(28854,1)
mri_ssp585_train_ECs_x3 = tf.convert_to_tensor(mri_ssp585_train_ECs_l3)
mri_ssp585_train_ECs_x3=tf.cast(mri_ssp585_train_ECs_x3,tf.float64)
mri_ssp585_train_ECs_l4 = mri_ssp585_train_ECs[:,3].reshape(28854,1)
mri_ssp585_train_ECs_x4 = tf.convert_to_tensor(mri_ssp585_train_ECs_l4)
mri_ssp585_train_ECs_x4=tf.cast(mri_ssp585_train_ECs_x4,tf.float64)

In [46]:
mri_pred_ECs_ssp126 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_lb_ssp126 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_ub_ssp126 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_ssp370 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_lb_ssp370 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_ub_ssp370 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_ssp585 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_lb_ssp585 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_ub_ssp585 = np.zeros((28854,n_input_ECs))
scaler_x_ssp126 = StandardScaler()
mri_train_ssp126 = np.c_[mri_ssp126_train_ECs_x1,mri_ssp126_train_ECs_x2,mri_ssp126_train_ECs_x3,mri_ssp126_train_ECs_x4]
mri_train_ssp126_sc = scaler_x_ssp126.fit_transform(mri_train_ssp126)
scaler_x_ssp370 = StandardScaler()
mri_train_ssp370 = np.c_[mri_ssp370_train_ECs_x1,mri_ssp370_train_ECs_x2,mri_ssp370_train_ECs_x3,mri_ssp370_train_ECs_x4]
mri_train_ssp370_sc = scaler_x_ssp370.fit_transform(mri_train_ssp370)
scaler_x_ssp585 = StandardScaler()
mri_train_ssp585 = np.c_[mri_ssp585_train_ECs_x1,mri_ssp585_train_ECs_x2,mri_ssp585_train_ECs_x3,mri_ssp585_train_ECs_x4]
mri_train_ssp585_sc = scaler_x_ssp585.fit_transform(mri_train_ssp585)
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

In [47]:
#EOF1

for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mri_train_sc[:,0].min(), mri_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(mri_train_sc[:, j].min(), mri_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mri_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mri_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mri_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_mri_eof[i].training_loss, model_mri_eof[i].trainable_variables)
    gf.utilities.print_summary(model_mri_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_mri_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_mri_eof1_timeperiod:',opt_mri_eof1_timeperiod)
    y_pred_mean_ssp126, y_pred_var_ssp126= model_mri_eof[i].predict_y(mri_train_ssp126_sc, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    mri_pred_mean_ssp126 = scaler_y.inverse_transform(y_pred_mean_ssp126)
    mri_pred_ECs_ssp126[:,i] =np.squeeze(mri_pred_mean_ssp126)
    print(mri_pred_ECs_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    mri_pred_ECs_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    mri_pred_ECs_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_ssp370, y_pred_var_ssp370= model_mri_eof[i].predict_y(mri_train_ssp370_sc, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    mri_pred_mean_ssp370 = scaler_y.inverse_transform(y_pred_mean_ssp370)
    mri_pred_ECs_ssp370[:,i] =np.squeeze(mri_pred_mean_ssp370)
    print(mri_pred_ECs_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    mri_pred_ECs_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    mri_pred_ECs_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_ssp585, y_pred_var_ssp585= model_mri_eof[i].predict_y(mri_train_ssp585_sc, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    mri_pred_mean_ssp585 = scaler_y.inverse_transform(y_pred_mean_ssp585)
    mri_pred_ECs_ssp585[:,i] =np.squeeze(mri_pred_mean_ssp585)
    print(mri_pred_ECs_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    mri_pred_ECs_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    mri_pred_ECs_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))
    y_pred_mean, y_pred_var = model_mri_eof[i].predict_y(mri_train_sc, full_cov=False)
    mri_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mri_pred_mean) / np.mean(obs_train)
    y = (np.std(mri_pred_mean) / np.mean(mri_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.79516
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.61161
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.21128
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.54576, -4.26863, -6.50453..."


The  1  ECs training for obsdata
opt_mri_eof1_timeperiod: 4179.094000000041
tf.Tensor(
[[0.78649224]
 [0.74008694]
 [0.67433541]
 ...
 [0.70406132]
 [0.71168664]
 [0.71843616]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-47374.86282624 -53834.55430649 -62987.27195485 ... -58849.37741933
 -57787.92143905 -56848.37746672]
tf.Tensor(
[[0.66660988]
 [0.65102183]
 [0.60870578]
 ...
 [0.79528675]
 [0.80394138]
 [0.80657558]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-64062.6782009  -66232.55893249 -72123.02061375 ... -46150.65278792
 -44945.91421513 -44579.22896985]
tf.Tensor(
[[0.66955994]
 [0.60232829]
 [0.51982827]
 ...
 [0.52220749]
 [0.52592056]
 [0.5285386 ]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-63652.02410385 -73010.77770204 -84494.911

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.12887
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.56676
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.4045
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.73077, -4.24251, -6.53212..."


The  2  ECs training for obsdata
opt_mri_eof1_timeperiod: 903.9070000000065
tf.Tensor(
[[-0.09264171]
 [ 0.00853298]
 [ 0.11227111]
 ...
 [ 0.37750448]
 [ 0.37080808]
 [ 0.36472775]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[-1961.59861772  -227.02265016  1551.50186966 ...  6098.76001752
  5983.95460597  5879.711088  ]
tf.Tensor(
[[0.16813726]
 [0.22692003]
 [0.30275841]
 ...
 [0.07295278]
 [0.06889824]
 [0.07373854]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[2509.29171659 3517.08509209 4817.28610879 ...  877.41406955  807.90156372
  890.88537094]
tf.Tensor(
[[0.35118791]
 [0.44791656]
 [0.52780897]
 ...
 [0.63336281]
 [0.64291442]
 [0.65728999]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 5647.57914275  7305.93067     8675.

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.63838
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.4565
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.70741
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.49815, -4.21643, -6.73137..."


The  3  ECs training for obsdata
opt_mri_eof1_timeperiod: 864.030999999959
tf.Tensor(
[[-0.04010289]
 [-0.07491781]
 [-0.11783488]
 ...
 [ 0.08288153]
 [ 0.09151265]
 [ 0.09895443]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ 702.30968161  266.97094902 -269.67997376 ... 2240.1524997  2348.07928112
 2441.13407221]
tf.Tensor(
[[-0.10519536]
 [-0.1172391 ]
 [-0.14515559]
 ...
 [ 0.03952515]
 [ 0.0449646 ]
 [ 0.05045742]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[-111.63069031 -262.23006692 -611.30829771 ... 1698.00821656 1766.02520269
 1834.70944473]
tf.Tensor(
[[ 0.10904769]
 [ 0.0295832 ]
 [-0.08829353]
 ...
 [ 0.0795603 ]
 [ 0.11068215]
 [ 0.13517002]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[2567.34393679 1573.69037384   99.71607835 ... 219

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.16992
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.55882
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.61461
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.7117, -4.07452, -6.59177..."


The  4  ECs training for obsdata
opt_mri_eof1_timeperiod: 18670.782000000007
tf.Tensor(
[[-0.18633435]
 [-0.06724291]
 [ 0.08989923]
 ...
 [ 0.27163286]
 [ 0.25640856]
 [ 0.24327078]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[ 293.01398945 1307.99196171 2647.26364679 ... 4196.12071798 4066.3689871
 3954.39988981]
tf.Tensor(
[[ 0.09542925]
 [ 0.15806584]
 [ 0.26448307]
 ...
 [-0.04105194]
 [-0.05696363]
 [-0.05979804]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[2694.39427675 3228.22571838 4135.18542628 ... 1531.20911232 1395.59894092
 1371.44213604]
tf.Tensor(
[[0.26823964]
 [0.38989519]
 [0.53240768]
 ...
 [0.64307958]
 [0.64469224]
 [0.65030995]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[4167.20146291 5204.03242495 6418.620

In [48]:
#合并校正后的两个主成分-
mri_ssp126_train_ECs_SGPRcor=tf.concat([mri_pred_ECs_ssp126[:,0].reshape(28854, 1),mri_pred_ECs_ssp126[:,1].reshape(28854, 1),mri_pred_ECs_ssp126[:,2].reshape(28854, 1),mri_pred_ECs_ssp126[:,3].reshape(28854, 1)],axis=1)
mri_ssp126_train_ECs_lb_SGPRcor=tf.concat([mri_pred_ECs_lb_ssp126[:,0].reshape(28854, 1),mri_pred_ECs_lb_ssp126[:,1].reshape(28854, 1),mri_pred_ECs_lb_ssp126[:,2].reshape(28854, 1),mri_pred_ECs_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
mri_ssp126_train_ECs_ub_SGPRcor=tf.concat([mri_pred_ECs_ub_ssp126[:,0].reshape(28854, 1),mri_pred_ECs_ub_ssp126[:,1].reshape(28854, 1),mri_pred_ECs_ub_ssp126[:,2].reshape(28854, 1),mri_pred_ECs_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
mri_ssp370_train_ECs_SGPRcor=tf.concat([mri_pred_ECs_ssp370[:,0].reshape(28854, 1),mri_pred_ECs_ssp370[:,1].reshape(28854, 1),mri_pred_ECs_ssp370[:,2].reshape(28854, 1),mri_pred_ECs_ssp370[:,3].reshape(28854, 1)],axis=1)
mri_ssp370_train_ECs_lb_SGPRcor=tf.concat([mri_pred_ECs_lb_ssp370[:,0].reshape(28854, 1),mri_pred_ECs_lb_ssp370[:,1].reshape(28854, 1),mri_pred_ECs_lb_ssp370[:,2].reshape(28854, 1),mri_pred_ECs_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
mri_ssp370_train_ECs_ub_SGPRcor=tf.concat([mri_pred_ECs_ub_ssp370[:,0].reshape(28854, 1),mri_pred_ECs_ub_ssp370[:,1].reshape(28854, 1),mri_pred_ECs_ub_ssp370[:,2].reshape(28854, 1),mri_pred_ECs_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
mri_ssp585_train_ECs_SGPRcor=tf.concat([mri_pred_ECs_ssp585[:,0].reshape(28854, 1),mri_pred_ECs_ssp585[:,1].reshape(28854, 1),mri_pred_ECs_ssp585[:,2].reshape(28854, 1),mri_pred_ECs_ssp585[:,3].reshape(28854, 1)],axis=1)
mri_ssp585_train_ECs_lb_SGPRcor=tf.concat([mri_pred_ECs_lb_ssp585[:,0].reshape(28854, 1),mri_pred_ECs_lb_ssp585[:,1].reshape(28854, 1),mri_pred_ECs_lb_ssp585[:,2].reshape(28854, 1),mri_pred_ECs_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
mri_ssp585_train_ECs_ub_SGPRcor=tf.concat([mri_pred_ECs_ub_ssp585[:,0].reshape(28854, 1),mri_pred_ECs_ub_ssp585[:,1].reshape(28854, 1),mri_pred_ECs_ub_ssp585[:,2].reshape(28854, 1),mri_pred_ECs_ub_ssp585[:,3].reshape(28854, 1)],axis=1)



In [51]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
mri_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
mri_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
mri_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_mri_eof_2GP = list()
mri_pred_ECs_range = np.zeros((13149,n_input_ECs))
mri_pred_ECs_2GP_ssp126 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_lb_ssp126 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_ub_ssp126 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_range_ssp126 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_ssp370 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_lb_ssp370 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_ub_ssp370 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_range_ssp370 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_ssp585 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_lb_ssp585 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_2GP_ub_ssp585 = np.zeros((28854,n_input_ECs))
mri_pred_ECs_range_ssp585 = np.zeros((28854,n_input_ECs))
for i in range(n_input_ECs):
    mri_pred_ECs_range[:,i] = np.subtract(mri_pred_ECs_ub[:,i],mri_pred_ECs_lb[:,i])
    mri_train_2GP = np.c_[mri_pred_ECs[:,i],mri_pred_ECs_lb[:,i],mri_pred_ECs_ub[:,i],mri_pred_ECs_range[:,i] ]
    mri_train_2GP_sc = scaler_z.fit_transform(mri_train_2GP)
    mri_pred_ECs_range_ssp126[:,i] = np.subtract(mri_pred_ECs_ub_ssp126[:,i],mri_pred_ECs_lb_ssp126[:,i])
    mri_train_2GP_ssp126 = np.c_[mri_pred_ECs_ssp126[:,i],mri_pred_ECs_lb_ssp126[:,i],mri_pred_ECs_ub_ssp126[:,i],mri_pred_ECs_range_ssp126[:,i] ]
    mri_train_2GP_sc_ssp126 = scaler_z.fit_transform(mri_train_2GP_ssp126 )
    mri_pred_ECs_range_ssp370[:,i] = np.subtract(mri_pred_ECs_ub_ssp370[:,i],mri_pred_ECs_lb_ssp370[:,i])
    mri_train_2GP_ssp370 = np.c_[mri_pred_ECs_ssp370[:,i],mri_pred_ECs_lb_ssp370[:,i],mri_pred_ECs_ub_ssp370[:,i],mri_pred_ECs_range_ssp370[:,i] ]
    mri_train_2GP_sc_ssp370 = scaler_z.fit_transform(mri_train_2GP_ssp370 )
    mri_pred_ECs_range_ssp585[:,i] = np.subtract(mri_pred_ECs_ub_ssp585[:,i],mri_pred_ECs_lb_ssp585[:,i])
    mri_train_2GP_ssp585 = np.c_[mri_pred_ECs_ssp585[:,i],mri_pred_ECs_lb_ssp585[:,i],mri_pred_ECs_ub_ssp585[:,i],mri_pred_ECs_range_ssp585[:,i] ]
    mri_train_2GP_sc_ssp585 = scaler_z.fit_transform(mri_train_2GP_ssp585 )
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(mri_train_2GP_sc[:,0].min(), mri_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(mri_train_2GP_sc[:, j].min(), mri_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(mri_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (mri_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_mri_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_mri_eof_2GP[i].training_loss, model_mri_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_mri_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_mri_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_mri_eof1_2GP_timeperiod:',opt_mri_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_mri_eof_2GP[i].predict_y(mri_train_2GP_sc, full_cov=False)
    mri_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(mri_pred_mean) / np.mean(obs_train)
    y = (np.std(mri_pred_mean) / np.mean(mri_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    mri_pred_ECs_2GP[:,i] =np.squeeze(mri_pred_mean)
    print(mri_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    mri_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    mri_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    y_pred_mean_2GP_ssp126, y_pred_var_2GP_ssp126= model_mri_eof_2GP[i].predict_y(mri_train_2GP_sc_ssp126, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    mri_pred_mean_2GP_ssp126 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp126)
    mri_pred_ECs_2GP_ssp126[:,i] =np.squeeze(mri_pred_mean_2GP_ssp126)
    print(mri_pred_ECs_2GP_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    mri_pred_ECs_2GP_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    mri_pred_ECs_2GP_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_2GP_ssp370, y_pred_var_2GP_ssp370= model_mri_eof_2GP[i].predict_y(mri_train_2GP_sc_ssp370, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    mri_pred_mean_2GP_ssp370 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp370)
    mri_pred_ECs_2GP_ssp370[:,i] =np.squeeze(mri_pred_mean_2GP_ssp370)
    print(mri_pred_ECs_2GP_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    mri_pred_ECs_2GP_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    mri_pred_ECs_2GP_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_2GP_ssp585, y_pred_var_2GP_ssp585= model_mri_eof_2GP[i].predict_y(mri_train_2GP_sc_ssp585, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    mri_pred_mean_2GP_ssp585 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp585)
    mri_pred_ECs_2GP_ssp585[:,i] =np.squeeze(mri_pred_mean_2GP_ssp585)
    print(mri_pred_ECs_2GP_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    mri_pred_ECs_2GP_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    mri_pred_ECs_2GP_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,115.00835
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,30.20261
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.19235
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.57054, -3.21544, -3.4176..."


The  1  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 18670.782000000007
[-38322.11157351 -37399.96173039 -36369.11848015 ... -38183.09731034
 -38032.52304639 -37812.16324926]
0.9999981319487102 0.8986403090202538
tf.Tensor(
[[-0.18633435]
 [-0.06724291]
 [ 0.08989923]
 ...
 [ 0.27163286]
 [ 0.25640856]
 [ 0.24327078]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-44610.39075089 -50704.43062745 -59398.83026594 ... -55477.99691713
 -54468.42379963 -53575.88641296]
tf.Tensor(
[[ 0.09542925]
 [ 0.15806584]
 [ 0.26448307]
 ...
 [-0.04105194]
 [-0.05696363]
 [-0.05979804]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-54507.18386925 -56598.6101593  -62299.53662868 ... -37414.47915145
 -36274.58458781 -35927.74377112]
tf.Tensor(
[[0.26823964]
 [0.38989519]
 [0.53240768]
 ...
 [0.64307958]
 [0.64469224]
 [0.65030995]], shape=(2

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,332.74705
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,75.45329
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.36423
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-3.77815, -5.36173, -2.58469..."


The  2  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 18670.782000000007
[-1134.73645687 -1163.07421085 -1229.83565488 ...   397.25222735
   372.34507818   352.33918853]
1.0000422396797122 0.7972338025427492
tf.Tensor(
[[-0.18633435]
 [-0.06724291]
 [ 0.08989923]
 ...
 [ 0.27163286]
 [ 0.25640856]
 [ 0.24327078]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[-3194.77382116 -1382.27483771   481.41382207 ...  5262.66935771
  5141.58873732  5031.64948245]
tf.Tensor(
[[ 0.09542925]
 [ 0.15806584]
 [ 0.26448307]
 ...
 [-0.04105194]
 [-0.05696363]
 [-0.05979804]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 881.34507596 1983.11239346 3407.68044114 ... -902.10937627 -977.87769447
 -887.4828763 ]
tf.Tensor(
[[0.26823964]
 [0.38989519]
 [0.53240768]
 ...
 [0.64307958]
 [0.64469224]
 [0.65030995]], shape=(28854, 1), dt

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,7.27409
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,7.04961
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.61901
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.67049, -5.79394, -6.17846..."


The  3  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 18670.782000000007
[3214.48104515 3196.23385    3168.57087134 ... 2867.95807777 2801.36155202
 2747.84274198]
1.0001174695348087 0.616731706791308
tf.Tensor(
[[-0.18633435]
 [-0.06724291]
 [ 0.08989923]
 ...
 [ 0.27163286]
 [ 0.25640856]
 [ 0.24327078]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ 858.01586777  345.6518575  -280.38156675 ... 2601.03780294 2724.20527739
 2829.98644888]
tf.Tensor(
[[ 0.09542925]
 [ 0.15806584]
 [ 0.26448307]
 ...
 [-0.04105194]
 [-0.05696363]
 [-0.05979804]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[  38.51590758 -136.94576926 -532.16721243 ... 2061.04492041 2135.62178511
 2212.86394552]
tf.Tensor(
[[0.26823964]
 [0.38989519]
 [0.53240768]
 ...
 [0.64307958]
 [0.64469224]
 [0.65030995]], shape=(28854, 1), dtype=float64) [[0.01163053

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,515.70226
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,113.59514
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.56486
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-0.97261, -6.51658, -10.15603..."


The  4  ECs training for obsdata
opt_mri_eof1_2GP_timeperiod: 18670.782000000007
[495.77349754 341.15330078 164.23736475 ... 499.09175459 444.60193492
 396.04975781]
1.0000053559816298 0.6595088695656534
tf.Tensor(
[[-0.18633435]
 [-0.06724291]
 [ 0.08989923]
 ...
 [ 0.27163286]
 [ 0.25640856]
 [ 0.24327078]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[-864.88122088  309.14440682 1857.44720543 ... 3639.22085296 3489.79766134
 3360.82779785]
tf.Tensor(
[[ 0.09542925]
 [ 0.15806584]
 [ 0.26448307]
 ...
 [-0.04105194]
 [-0.05696363]
 [-0.05979804]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[1585.15646183 2207.8490399  3265.36695313 ...  225.25119417   67.00287243
   38.83572554]
tf.Tensor(
[[0.26823964]
 [0.38989519]
 [0.53240768]
 ...
 [0.64307958]
 [0.64469224]
 [0.65030995]], shape=(28854, 1), dtype=float64) [[-0.

In [52]:
#合并校正后的两个主成分-
mri_ssp126_train_ECs_2GP_SGPRcor=tf.concat([mri_pred_ECs_2GP_ssp126[:,0].reshape(28854, 1),mri_pred_ECs_2GP_ssp126[:,1].reshape(28854, 1),mri_pred_ECs_2GP_ssp126[:,2].reshape(28854, 1),mri_pred_ECs_2GP_ssp126[:,3].reshape(28854, 1)],axis=1)
mri_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([mri_pred_ECs_2GP_lb_ssp126[:,0].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp126[:,1].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp126[:,2].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
mri_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([mri_pred_ECs_2GP_ub_ssp126[:,0].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp126[:,1].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp126[:,2].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
mri_ssp370_train_ECs_2GP_SGPRcor=tf.concat([mri_pred_ECs_2GP_ssp370[:,0].reshape(28854, 1),mri_pred_ECs_2GP_ssp370[:,1].reshape(28854, 1),mri_pred_ECs_2GP_ssp370[:,2].reshape(28854, 1),mri_pred_ECs_2GP_ssp370[:,3].reshape(28854, 1)],axis=1)
mri_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([mri_pred_ECs_2GP_lb_ssp370[:,0].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp370[:,1].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp370[:,2].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
mri_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([mri_pred_ECs_2GP_ub_ssp370[:,0].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp370[:,1].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp370[:,2].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
mri_ssp585_train_ECs_2GP_SGPRcor=tf.concat([mri_pred_ECs_2GP_ssp585[:,0].reshape(28854, 1),mri_pred_ECs_2GP_ssp585[:,1].reshape(28854, 1),mri_pred_ECs_2GP_ssp585[:,2].reshape(28854, 1),mri_pred_ECs_2GP_ssp585[:,3].reshape(28854, 1)],axis=1)
mri_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([mri_pred_ECs_2GP_lb_ssp585[:,0].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp585[:,1].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp585[:,2].reshape(28854, 1),mri_pred_ECs_2GP_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
mri_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([mri_pred_ECs_2GP_ub_ssp585[:,0].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp585[:,1].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp585[:,2].reshape(28854, 1),mri_pred_ECs_2GP_ub_ssp585[:,3].reshape(28854, 1)],axis=1)

# 写入Excel文件
df_mean_merge_2GP_ssp126 = pd.DataFrame(mri_ssp126_train_ECs_2GP_SGPRcor,columns = ['mri_EC1_2GP','mri_EC2_2GP','mri_EC3_2GP','mri_EC4_2GP'])
df_lb_merge_2GP_ssp126 = pd.DataFrame(mri_ssp126_train_ECs_2GP_lb_SGPRcor,columns = ['mri_EC1_2GP_lb','mri_EC2_2GP_lb','mri_EC3_2GP_lb','mri_EC4_2GP_lb'])
df_ub_merge_2GP_ssp126 = pd.DataFrame(mri_ssp126_train_ECs_2GP_ub_SGPRcor,columns = ['mri_EC1_2GP_ub','mri_EC2_2GP_ub','mri_EC3_2GP_ub','mri_EC4_2GP_ub'])
df_mean_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean_ssp126_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb_ssp126_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub_ssp126_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp370 = pd.DataFrame(mri_ssp370_train_ECs_2GP_SGPRcor,columns = ['mri_EC1_2GP','mri_EC2_2GP','mri_EC3_2GP','mri_EC4_2GP'])
df_lb_merge_2GP_ssp370 = pd.DataFrame(mri_ssp370_train_ECs_2GP_lb_SGPRcor,columns = ['mri_EC1_2GP_lb','mri_EC2_2GP_lb','mri_EC3_2GP_lb','mri_EC4_2GP_lb'])
df_ub_merge_2GP_ssp370 = pd.DataFrame(mri_ssp370_train_ECs_2GP_ub_SGPRcor,columns = ['mri_EC1_2GP_ub','mri_EC2_2GP_ub','mri_EC3_2GP_ub','mri_EC4_2GP_ub'])
df_mean_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean_ssp370_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb_ssp370_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub_ssp370_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp585 = pd.DataFrame(mri_ssp585_train_ECs_2GP_SGPRcor,columns = ['mri_EC1_2GP','mri_EC2_2GP','mri_EC3_2GP','mri_EC4_2GP'])
df_lb_merge_2GP_ssp585 = pd.DataFrame(mri_ssp585_train_ECs_2GP_lb_SGPRcor,columns = ['mri_EC1_2GP_lb','mri_EC2_2GP_lb','mri_EC3_2GP_lb','mri_EC4_2GP_lb'])
df_ub_merge_2GP_ssp585 = pd.DataFrame(mri_ssp585_train_ECs_2GP_ub_SGPRcor,columns = ['mri_EC1_2GP_ub','mri_EC2_2GP_ub','mri_EC3_2GP_ub','mri_EC4_2GP_ub'])
df_mean_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean_ssp585_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb_ssp585_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub_ssp585_2GP.xlsx',index=False)

In [4]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean_ssp126_2GP.xlsx')
# 获取指定列的数据
mri_ssp126_train_SGPRcor_EC1_2GP = data_frame['mri_EC1_2GP']
mri_ssp126_train_SGPRcor_EC1_2GP_array = np.array(mri_ssp126_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC1_2GP_array)
mri_ssp126_train_SGPRcor_EC1_2GP_tf=tf.cast(mri_ssp126_train_SGPRcor_EC1_2GP_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC2_2GP = data_frame['mri_EC2_2GP']
mri_ssp126_train_SGPRcor_EC2_2GP_array = np.array(mri_ssp126_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC2_2GP_array)
mri_ssp126_train_SGPRcor_EC2_2GP_tf=tf.cast(mri_ssp126_train_SGPRcor_EC2_2GP_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC3_2GP = data_frame['mri_EC3_2GP']
mri_ssp126_train_SGPRcor_EC3_2GP_array = np.array(mri_ssp126_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC3_2GP_array)
mri_ssp126_train_SGPRcor_EC3_2GP_tf=tf.cast(mri_ssp126_train_SGPRcor_EC3_2GP_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC4_2GP = data_frame['mri_EC4_2GP']
mri_ssp126_train_SGPRcor_EC4_2GP_array = np.array(mri_ssp126_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC4_2GP_array)
mri_ssp126_train_SGPRcor_EC4_2GP_tf=tf.cast(mri_ssp126_train_SGPRcor_EC4_2GP_tf,tf.float64)
mri_ssp126_train_ECs_2GP_SGPRcor=tf.concat([mri_ssp126_train_SGPRcor_EC1_2GP_tf,mri_ssp126_train_SGPRcor_EC2_2GP_tf,mri_ssp126_train_SGPRcor_EC3_2GP_tf,mri_ssp126_train_SGPRcor_EC4_2GP_tf],axis=1)
print(mri_ssp126_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean_ssp370_2GP.xlsx')
# 获取指定列的数据
mri_ssp370_train_SGPRcor_EC1_2GP = data_frame['mri_EC1_2GP']
mri_ssp370_train_SGPRcor_EC1_2GP_array = np.array(mri_ssp370_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC1_2GP_array)
mri_ssp370_train_SGPRcor_EC1_2GP_tf=tf.cast(mri_ssp370_train_SGPRcor_EC1_2GP_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC2_2GP = data_frame['mri_EC2_2GP']
mri_ssp370_train_SGPRcor_EC2_2GP_array = np.array(mri_ssp370_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC2_2GP_array)
mri_ssp370_train_SGPRcor_EC2_2GP_tf=tf.cast(mri_ssp370_train_SGPRcor_EC2_2GP_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC3_2GP = data_frame['mri_EC3_2GP']
mri_ssp370_train_SGPRcor_EC3_2GP_array = np.array(mri_ssp370_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC3_2GP_array)
mri_ssp370_train_SGPRcor_EC3_2GP_tf=tf.cast(mri_ssp370_train_SGPRcor_EC3_2GP_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC4_2GP = data_frame['mri_EC4_2GP']
mri_ssp370_train_SGPRcor_EC4_2GP_array = np.array(mri_ssp370_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC4_2GP_array)
mri_ssp370_train_SGPRcor_EC4_2GP_tf=tf.cast(mri_ssp370_train_SGPRcor_EC4_2GP_tf,tf.float64)
mri_ssp370_train_ECs_2GP_SGPRcor=tf.concat([mri_ssp370_train_SGPRcor_EC1_2GP_tf,mri_ssp370_train_SGPRcor_EC2_2GP_tf,mri_ssp370_train_SGPRcor_EC3_2GP_tf,mri_ssp370_train_SGPRcor_EC4_2GP_tf],axis=1)
print(mri_ssp370_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_mean_ssp585_2GP.xlsx')
# 获取指定列的数据
mri_ssp585_train_SGPRcor_EC1_2GP = data_frame['mri_EC1_2GP']
mri_ssp585_train_SGPRcor_EC1_2GP_array = np.array(mri_ssp585_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC1_2GP_array)
mri_ssp585_train_SGPRcor_EC1_2GP_tf=tf.cast(mri_ssp585_train_SGPRcor_EC1_2GP_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC2_2GP = data_frame['mri_EC2_2GP']
mri_ssp585_train_SGPRcor_EC2_2GP_array = np.array(mri_ssp585_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC2_2GP_array)
mri_ssp585_train_SGPRcor_EC2_2GP_tf=tf.cast(mri_ssp585_train_SGPRcor_EC2_2GP_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC3_2GP = data_frame['mri_EC3_2GP']
mri_ssp585_train_SGPRcor_EC3_2GP_array = np.array(mri_ssp585_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC3_2GP_array)
mri_ssp585_train_SGPRcor_EC3_2GP_tf=tf.cast(mri_ssp585_train_SGPRcor_EC3_2GP_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC4_2GP = data_frame['mri_EC4_2GP']
mri_ssp585_train_SGPRcor_EC4_2GP_array = np.array(mri_ssp585_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC4_2GP_array)
mri_ssp585_train_SGPRcor_EC4_2GP_tf=tf.cast(mri_ssp585_train_SGPRcor_EC4_2GP_tf,tf.float64)
mri_ssp585_train_ECs_2GP_SGPRcor=tf.concat([mri_ssp585_train_SGPRcor_EC1_2GP_tf,mri_ssp585_train_SGPRcor_EC2_2GP_tf,mri_ssp585_train_SGPRcor_EC3_2GP_tf,mri_ssp585_train_SGPRcor_EC4_2GP_tf],axis=1)
print(mri_ssp585_train_ECs_2GP_SGPRcor)

tf.Tensor(
[[-44610.39075089  -3194.77382116    858.01586777   -864.88122088]
 [-50704.43062745  -1382.27483771    345.6518575     309.14440682]
 [-59398.83026594    481.41382207   -280.38156675   1857.44720543]
 ...
 [-55477.99691713   5262.66935771   2601.03780294   3639.22085296]
 [-54468.42379963   5141.58873732   2724.20527739   3489.79766134]
 [-53575.88641296   5031.64948245   2829.98644888   3360.82779785]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-5.45071839e+04  8.81345076e+02  3.85159076e+01  1.58515646e+03]
 [-5.65986102e+04  1.98311239e+03 -1.36945769e+02  2.20784904e+03]
 [-6.22995366e+04  3.40768044e+03 -5.32167212e+02  3.26536695e+03]
 ...
 [-3.74144792e+04 -9.02109376e+02  2.06104492e+03  2.25251194e+02]
 [-3.62745846e+04 -9.77877694e+02  2.13562179e+03  6.70028724e+01]
 [-3.59277438e+04 -8.87482876e+02  2.21286395e+03  3.88357255e+01]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-4.42267178e+04  2.10023966e+03  2.81445684e+03  2.20075510e+03]
 [-5.36689180e+0

In [5]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb_ssp126_2GP.xlsx')
# 获取指定列的数据
mri_ssp126_train_SGPRcor_EC1_2GP_lb = data_frame['mri_EC1_2GP_lb']
mri_ssp126_train_SGPRcor_EC1_2GP_lb_array = np.array(mri_ssp126_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC1_2GP_lb_array)
mri_ssp126_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(mri_ssp126_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC2_2GP_lb = data_frame['mri_EC2_2GP_lb']
mri_ssp126_train_SGPRcor_EC2_2GP_lb_array = np.array(mri_ssp126_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC2_2GP_lb_array)
mri_ssp126_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(mri_ssp126_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC3_2GP_lb = data_frame['mri_EC3_2GP_lb']
mri_ssp126_train_SGPRcor_EC3_2GP_lb_array = np.array(mri_ssp126_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC3_2GP_lb_array)
mri_ssp126_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(mri_ssp126_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC4_2GP_lb = data_frame['mri_EC4_2GP_lb']
mri_ssp126_train_SGPRcor_EC4_2GP_lb_array = np.array(mri_ssp126_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC4_2GP_lb_array)
mri_ssp126_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(mri_ssp126_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
mri_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([mri_ssp126_train_SGPRcor_EC1_2GP_lb_tf,mri_ssp126_train_SGPRcor_EC2_2GP_lb_tf,mri_ssp126_train_SGPRcor_EC3_2GP_lb_tf,mri_ssp126_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(mri_ssp126_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb_ssp370_2GP.xlsx')
# 获取指定列的数据
mri_ssp370_train_SGPRcor_EC1_2GP_lb = data_frame['mri_EC1_2GP_lb']
mri_ssp370_train_SGPRcor_EC1_2GP_lb_array = np.array(mri_ssp370_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC1_2GP_lb_array)
mri_ssp370_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(mri_ssp370_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC2_2GP_lb = data_frame['mri_EC2_2GP_lb']
mri_ssp370_train_SGPRcor_EC2_2GP_lb_array = np.array(mri_ssp370_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC2_2GP_lb_array)
mri_ssp370_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(mri_ssp370_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC3_2GP_lb = data_frame['mri_EC3_2GP_lb']
mri_ssp370_train_SGPRcor_EC3_2GP_lb_array = np.array(mri_ssp370_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC3_2GP_lb_array)
mri_ssp370_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(mri_ssp370_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC4_2GP_lb = data_frame['mri_EC4_2GP_lb']
mri_ssp370_train_SGPRcor_EC4_2GP_lb_array = np.array(mri_ssp370_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC4_2GP_lb_array)
mri_ssp370_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(mri_ssp370_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
mri_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([mri_ssp370_train_SGPRcor_EC1_2GP_lb_tf,mri_ssp370_train_SGPRcor_EC2_2GP_lb_tf,mri_ssp370_train_SGPRcor_EC3_2GP_lb_tf,mri_ssp370_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(mri_ssp370_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_lb_ssp585_2GP.xlsx')
# 获取指定列的数据
mri_ssp585_train_SGPRcor_EC1_2GP_lb = data_frame['mri_EC1_2GP_lb']
mri_ssp585_train_SGPRcor_EC1_2GP_lb_array = np.array(mri_ssp585_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC1_2GP_lb_array)
mri_ssp585_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(mri_ssp585_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC2_2GP_lb = data_frame['mri_EC2_2GP_lb']
mri_ssp585_train_SGPRcor_EC2_2GP_lb_array = np.array(mri_ssp585_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC2_2GP_lb_array)
mri_ssp585_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(mri_ssp585_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC3_2GP_lb = data_frame['mri_EC3_2GP_lb']
mri_ssp585_train_SGPRcor_EC3_2GP_lb_array = np.array(mri_ssp585_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC3_2GP_lb_array)
mri_ssp585_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(mri_ssp585_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC4_2GP_lb = data_frame['mri_EC4_2GP_lb']
mri_ssp585_train_SGPRcor_EC4_2GP_lb_array = np.array(mri_ssp585_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC4_2GP_lb_array)
mri_ssp585_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(mri_ssp585_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
mri_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([mri_ssp585_train_SGPRcor_EC1_2GP_lb_tf,mri_ssp585_train_SGPRcor_EC2_2GP_lb_tf,mri_ssp585_train_SGPRcor_EC3_2GP_lb_tf,mri_ssp585_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(mri_ssp585_train_ECs_2GP_lb_SGPRcor)

tf.Tensor(
[[-397246.30718313  -29980.32111892  -20390.31868845  -12836.92834394]
 [-380758.26241924  -27949.61817423  -18909.2104087   -11827.44072335]
 [-359221.17842536  -25297.06473248  -16974.55084888  -10508.82350786]
 ...
 [-333363.41630909  -22112.36730932  -14651.76812268   -8925.67091904]
 [-335481.01945321  -22373.17585285  -14841.99077157   -9055.32208083]
 [-337307.4421213   -22598.12198463  -15006.05690401   -9167.14559294]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-358574.11287672  -25217.37075215  -16916.42545224  -10469.20664179]
 [-350098.77768559  -24173.53029828  -16155.09266203   -9950.30060543]
 [-335787.22900374  -22410.88927763  -14869.49733812   -9074.06989205]
 ...
 [-376767.64711431  -27458.12543711  -18550.73652934  -11583.11358774]
 [-378956.87211826  -27727.75508245  -18747.39291438  -11717.14982866]
 [-379350.88391043  -27776.28241957  -18782.78668824  -11741.27336982]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-335158.42678651  -22333.44464835

In [6]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub_ssp126_2GP.xlsx')
# 获取指定列的数据
mri_ssp126_train_SGPRcor_EC1_2GP_ub = data_frame['mri_EC1_2GP_ub']
mri_ssp126_train_SGPRcor_EC1_2GP_ub_array = np.array(mri_ssp126_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC1_2GP_ub_array)
mri_ssp126_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(mri_ssp126_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC2_2GP_ub = data_frame['mri_EC2_2GP_ub']
mri_ssp126_train_SGPRcor_EC2_2GP_ub_array = np.array(mri_ssp126_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC2_2GP_ub_array)
mri_ssp126_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(mri_ssp126_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC3_2GP_ub = data_frame['mri_EC3_2GP_ub']
mri_ssp126_train_SGPRcor_EC3_2GP_ub_array = np.array(mri_ssp126_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC3_2GP_ub_array)
mri_ssp126_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(mri_ssp126_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
mri_ssp126_train_SGPRcor_EC4_2GP_ub = data_frame['mri_EC4_2GP_ub']
mri_ssp126_train_SGPRcor_EC4_2GP_ub_array = np.array(mri_ssp126_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp126_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(mri_ssp126_train_SGPRcor_EC4_2GP_ub_array)
mri_ssp126_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(mri_ssp126_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
mri_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([mri_ssp126_train_SGPRcor_EC1_2GP_ub_tf,mri_ssp126_train_SGPRcor_EC2_2GP_ub_tf,mri_ssp126_train_SGPRcor_EC3_2GP_ub_tf,mri_ssp126_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(mri_ssp126_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub_ssp370_2GP.xlsx')
# 获取指定列的数据
mri_ssp370_train_SGPRcor_EC1_2GP_ub = data_frame['mri_EC1_2GP_ub']
mri_ssp370_train_SGPRcor_EC1_2GP_ub_array = np.array(mri_ssp370_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC1_2GP_ub_array)
mri_ssp370_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(mri_ssp370_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC2_2GP_ub = data_frame['mri_EC2_2GP_ub']
mri_ssp370_train_SGPRcor_EC2_2GP_ub_array = np.array(mri_ssp370_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC2_2GP_ub_array)
mri_ssp370_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(mri_ssp370_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC3_2GP_ub = data_frame['mri_EC3_2GP_ub']
mri_ssp370_train_SGPRcor_EC3_2GP_ub_array = np.array(mri_ssp370_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC3_2GP_ub_array)
mri_ssp370_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(mri_ssp370_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
mri_ssp370_train_SGPRcor_EC4_2GP_ub = data_frame['mri_EC4_2GP_ub']
mri_ssp370_train_SGPRcor_EC4_2GP_ub_array = np.array(mri_ssp370_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp370_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(mri_ssp370_train_SGPRcor_EC4_2GP_ub_array)
mri_ssp370_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(mri_ssp370_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
mri_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([mri_ssp370_train_SGPRcor_EC1_2GP_ub_tf,mri_ssp370_train_SGPRcor_EC2_2GP_ub_tf,mri_ssp370_train_SGPRcor_EC3_2GP_ub_tf,mri_ssp370_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(mri_ssp370_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\mri_ECs_pred_ub_ssp585_2GP.xlsx')
# 获取指定列的数据
mri_ssp585_train_SGPRcor_EC1_2GP_ub = data_frame['mri_EC1_2GP_ub']
mri_ssp585_train_SGPRcor_EC1_2GP_ub_array = np.array(mri_ssp585_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC1_2GP_ub_array)
mri_ssp585_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(mri_ssp585_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC2_2GP_ub = data_frame['mri_EC2_2GP_ub']
mri_ssp585_train_SGPRcor_EC2_2GP_ub_array = np.array(mri_ssp585_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC2_2GP_ub_array)
mri_ssp585_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(mri_ssp585_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC3_2GP_ub = data_frame['mri_EC3_2GP_ub']
mri_ssp585_train_SGPRcor_EC3_2GP_ub_array = np.array(mri_ssp585_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC3_2GP_ub_array)
mri_ssp585_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(mri_ssp585_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
mri_ssp585_train_SGPRcor_EC4_2GP_ub = data_frame['mri_EC4_2GP_ub']
mri_ssp585_train_SGPRcor_EC4_2GP_ub_array = np.array(mri_ssp585_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
mri_ssp585_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(mri_ssp585_train_SGPRcor_EC4_2GP_ub_array)
mri_ssp585_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(mri_ssp585_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
mri_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([mri_ssp585_train_SGPRcor_EC1_2GP_ub_tf,mri_ssp585_train_SGPRcor_EC2_2GP_ub_tf,mri_ssp585_train_SGPRcor_EC3_2GP_ub_tf,mri_ssp585_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(mri_ssp585_train_ECs_2GP_ub_SGPRcor)

tf.Tensor(
[[31658.56004777 22844.52193235 18137.87305174 13422.95632283]
 [48325.95354738 24897.31385186 19635.09208939 14443.42464677]
 [70537.73986    27632.9650778  21630.35963548 15803.35080144]
 ...
 [95275.19892669 30679.68357536 23852.50640719 17317.912355  ]
 [93154.31057598 30418.47041875 23661.98865074 17188.06005502]
 [91323.13219864 30192.93856362 23497.49531661 17075.94537257]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[71430.25126232 27742.88869572 21710.53324288 15857.99519529]
 [80393.14220979 28846.7775619  22515.66278874 16406.75204219]
 [95708.48930737 30733.04854765 23891.4285462  17344.44074461]
 ...
 [51626.98659183 25303.87615927 19931.62132885 14645.53181237]
 [49386.34731468 25027.9142218  19730.3464395  14508.34771051]
 [48991.24892424 24979.25305692 19694.85505737 14484.1576419 ]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[ 96125.52823621  30784.41195629  23928.89082989  17369.9741321 ]
 [113828.63172756  32964.76413902  25519.14689324  18453.8542450

In [7]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((28854,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [8]:
recondata_SGPR_mri_mix4_2GP_ssp126 = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp126_train_ECs_2GP_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp370 = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp370_train_ECs_2GP_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp585 = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp585_train_ECs_2GP_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp126_lb = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp126_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp370_lb = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp370_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp585_lb = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp585_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp126_ub = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp126_train_ECs_2GP_ub_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp370_ub = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp370_train_ECs_2GP_ub_SGPRcor)
recondata_SGPR_mri_mix4_2GP_ssp585_ub = ReconstructDataFromEOFandECs(obsdata_eofs,mri_ssp585_train_ECs_2GP_ub_SGPRcor)

In [9]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2100, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 2022-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [10]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc1 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp126.nc' 
out_file_nc2 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp370.nc' 
out_file_nc3 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp585.nc'
out_file_nc4 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp126_lb.nc' 
out_file_nc5 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp370_lb.nc' 
out_file_nc6 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp585_lb.nc' 
out_file_nc7 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp126_ub.nc' 
out_file_nc8 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp370_ub.nc' 
out_file_nc9 = out_dir+'recondata_SGPR_mri_mix4_2GP_ssp585_ub.nc' 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp126,out_file_nc1) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp370,out_file_nc2) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp585,out_file_nc3) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp126_lb,out_file_nc4) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp370_lb,out_file_nc5) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp585_lb,out_file_nc6) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp126_ub,out_file_nc7) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp370_ub,out_file_nc8) 
writeNetCDF(recondata_SGPR_mri_mix4_2GP_ssp585_ub,out_file_nc9) 